In [4]:
# IB-MULTI-STRATEGY-ATS/broker/functions.py
import numpy as np
import pandas as pd
import yfinance as yf
from ib_insync import *
# from gui.log import add_log
import datetime

util.startLoop()  # Needed in script mode
ib = IB()
try:
    ib.connect('127.0.0.1', 7497, clientId=0)
except ConnectionError:
    print('Connection failed. Start TWS or IB Gateway and try again!')
except:
    ib.connect('127.0.0.1', 7497, clientId=1)


In [20]:
future_symbol = "ES"
index_symbol = "SPX"

In [38]:
today = datetime.datetime.now()

# Generate contract months for the next 9 maturities
monthly_expirations = [
    f"{today.year + (today.month + i - 1) // 12}{(today.month + i - 1) % 12 + 1:02}" 
    for i in range(12)
]

# Create Future objects for each contract month
contracts = [Future(future_symbol, lastTradeDateOrContractMonth=exp) for exp in monthly_expirations]
qualified_contracts = ib.qualifyContracts(*contracts)
ib.sleep(1)

# Set market data type to delayed frozen data
ib.reqMarketDataType(3)

idx = Index(index_symbol)
ib.qualifyContracts(idx)
idx_details = ib.reqMktData(idx)
ib.sleep(1)
spot = idx_details.last

futures_data = {
    'Contract': ["Index"],
    'LastPrice': [spot],
    'DTE': [0],
    'AnnualizedYield': [None]
}

if not qualified_contracts:
    contracts = [Future(future_symbol, exchange="CME" ,lastTradeDateOrContractMonth=exp) for exp in monthly_expirations]
    qualified_contracts = ib.qualifyContracts(*contracts)

Error 200, reqId 284: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202312')
Error 200, reqId 285: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202401')
Error 200, reqId 286: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202402')
Error 200, reqId 288: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202404')
Error 200, reqId 289: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202405')
Error 200, reqId 291: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrContractMonth='202407')
Error 200, reqId 292: No security definition has been found for the request, contract: Future(symbol='ES', lastTradeDateOrCo

In [39]:
qualified_contracts
contract = qualified_contracts[0]

In [40]:
market_data = ib.reqMktData(contract)
ib.sleep(1)  # Wait for the data to be fetched
last_price = market_data.last if market_data.last > 0 else market_data.bid

expiration_date = datetime.datetime.strptime(contract.lastTradeDateOrContractMonth, '%Y%m%d')
dte = (expiration_date - today).days
annualized_yield = None

if dte != 0:
    annualized_yield = ((last_price / spot) ** (365 / dte) - 1) if last_price <= spot else ((spot / last_price) ** (365 / dte) - 1)

futures_data['Contract'].append(contract.localSymbol)
futures_data['LastPrice'].append(last_price)
futures_data['DTE'].append(dte)
futures_data['AnnualizedYield'].append(annualized_yield)

# Convert the dictionary to DataFrame
term_structure = pd.DataFrame(futures_data)

In [41]:
term_structure

,Contract,LastPrice,DTE,AnnualizedYield
0,Index,NaN,0,NaN
1,ESH4,NaN,77,NaN


Error 10197, reqId 283: No market data during competing live session, contract: Future(conId=533620665, symbol='ES', lastTradeDateOrContractMonth='20240315', multiplier='50', exchange='CME', currency='USD', localSymbol='ESH4', tradingClass='ES')
Error 10197, reqId 243: No market data during competing live session, contract: Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')
Error 10197, reqId 270: No market data during competing live session, contract: Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')
Error 10197, reqId 188: No market data during competing live session, contract: Index(conId=13455763, symbol='VIX', exchange='CBOE', currency='USD', localSymbol='VIX')
Error 10197, reqId 174: No market data during competing live session, contract: Future(conId=644107700, symbol='VXM', lastTradeDateOrContractMonth='20240117', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMF4', tradingClass='VXM')
Erro